<a href="https://colab.research.google.com/github/Avinandan22/ProbabilisticMachineLearning-PClub/blob/master/BMF_MCMC_RECOMMENDER_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender System Using BMF and MCMC
 
 In this problem ,we implement Bayesian Probabilistic Matrix Factorization using Markov Chain Monte Carlo

In [0]:
# Importing Train File
from google.colab import files
uploaded = files.upload()

Saving u1.base to u1.base


In [0]:
# Importing Test File
from google.colab import files
uploaded = files.upload()

Saving u1.test to u1.test


In [0]:
# Global Imports
import numpy as np
from scipy.stats import wishart

# Loading Data and Initializing Parameters

In [0]:
"""
    Function creating the NxM rating matrix from filename.
    It assumes that the file contains on every line a triple (user, movie, ranking).
"""

def ranking_matrix(N, M, filename, sep="	"):
    
    R = np.zeros((N, M))
    I = np.zeros((N, M))
    f = open(filename,"r")
    for line in f:
        if line[0] == '%':
            # this is a comment
            continue
        (user, movie, ranking,_) = line.split(sep)
        R[np.int(user) - 1, np.int(movie) - 1] = np.int(ranking)
        I[np.int(user) - 1, np.int(movie) - 1] = 1.0
    return R, I

In [0]:
# Setting K the number of latent features for every user and every movie as a hyperparameter
K = 50
N = 943
M = 1682

In [0]:
# Data Loading
R, I = ranking_matrix(N, M, "u1.base")
RTest, ITest = ranking_matrix(N ,M, "u1.test")

In [0]:
#Declaring the Hyper-parameters
mu0 = np.zeros(K)
uv0 = K
WU0 = np.eye(K)
mv0 = np.zeros(K)
vv0 = K
WV0 = np.eye(K)
beta0 = 1.0
alpha = 2.0

In [0]:
# Initializing lambdaU ,lambdaV ,muU ,muV
lambdaU = wishart.rvs(size = 1,df = uv0, scale = WU0)
muU = np.random.multivariate_normal(mu0, np.linalg.pinv(beta0 * lambdaU))
lambdaV = wishart.rvs(size = 1,df = vv0, scale = WV0)
muV = np.random.multivariate_normal(mv0, np.linalg.pinv(beta0 * lambdaV))

In [0]:
# Initializing U and V
U = np.zeros((N ,K))
V = np.zeros((M ,K))
U = np.random.multivariate_normal(muU ,lambdaU , N)
V = np.random.multivariate_normal(muV ,lambdaV , M)

In [0]:
# UUT and VVT created to stored updated values and avoid recalculation
VVT = np.zeros((M , K, K))
UUT = np.zeros((N , K, K))
for i in range(M):
  VVT[i] = V[i, :].reshape(K, 1) @ V[i ,:].reshape(1, K)
for i in range(N):
  UUT[i] = U[i, :].reshape(K, 1) @ U[i ,:].reshape(1,K)  

In [0]:
# sigmaUN and muUN contain the mean and covariance for the latent features of each user
# sigmaVM and muVM contain the mean and covariance for the latent features of each movie
sigmaUN = np.zeros((N, K, K))
sigmaVM = np.zeros((M, K, K))
muUN = np.zeros((N , K))
muVM = np.zeros((M, K))

#  Gibbs Sampling for Bayesian PMF 

In [0]:
T = 100
US = np.zeros((T ,N , K))
VS = np.zeros((T ,M ,K))
for t in range(T):
  
  #Calculating joint posterior for lambdaU and muU
  
  Ubar = np.sum(U ,axis = 0) / N
  SUbar = np.zeros((K ,K))
  for i in range(N):
    SUbar += U[i ,:].reshape(K ,1) @ U[i ,:].reshape(1 ,K)
  SUbar /= N
  mustar = (beta0 * mu0 + N * Ubar) / (beta0 + N)
  betastarU = beta0 + N
  uvstar = uv0 + N
  UWstar = np.linalg.pinv(np.linalg.pinv(WU0) + N * SUbar + beta0 * N * (mu0 - Ubar).reshape(K ,1) @ (mu0 - Ubar).reshape(1 ,K) / betastarU)
  
  # Sampling lambaU and muU
  
  lambdaU = wishart.rvs(size = 1,df = uvstar, scale = UWstar)
  muU = np.random.multivariate_normal(mustar, np.linalg.pinv(betastarU * lambdaU))
  
  #Calculating joint posterior for lambdaV and muV
  
  Vbar = np.sum(V ,axis = 0) / M
  SVbar = np.zeros((K , K))
  for j in range(M):
    SVbar += V[i ,:].reshape(K ,1) @ V[i ,:].reshape(1 ,K)
  SVbar /= M
  mvstar = (beta0 * mv0 + M * Vbar) / (beta0 + M)
  betastarV = beta0 + M
  vvstar = vv0 + M
  VWstar = np.linalg.pinv(np.linalg.pinv(WV0) + M * SVbar + beta0 * M * (mv0 - Vbar).reshape(K ,1) @ (mu0 - Vbar).reshape(1 ,K) / betastarV)
  
  # Sampling lambdaV and muV
  
  lambdaV = wishart.rvs(size = 1,df = vvstar, scale = VWstar)
  muV = np.random.multivariate_normal(mvstar, np.linalg.pinv(betastarV * lambdaV))
  
  #Calculating Conditional Posterior for U
  
  for i in range(N):
    A = np.zeros((K, K))
    B = np.zeros(K)
    for j in range(M):
      A += I[i][j] * VVT[j]
      B += R[i][j] * V[j ,:]
    B *= alpha
    sigmaUN[i] = np.linalg.pinv(lambdaU + alpha * A)
    muUN[i] = sigmaUN[i] @ ( B + lambdaU @ muU)
    
  #Sampling U
  
  for i in range(N):
    U[i] = np.random.multivariate_normal(muUN[i], sigmaUN[i])
  US[t] = U
  for i in range(N):
    UUT[i] = U[i, :].reshape(K, 1) @ U[i ,:].reshape(1,K)
    
  #Calculating Conditional Posterior for V
  
  for j in range(M):
    A = np.zeros((K ,K))
    B = np.zeros(K)
    for i in range(N):
      A += I[i][j] * UUT[i]
      B += R[i][j] * U[i ,:]
    B *= alpha
    sigmaVM[j] = np.linalg.pinv(lambdaV + alpha * A)
    muVM[j] = sigmaVM[j] @ (B + lambdaV @ muV)
    
  #Sampling V
  
  for j in range(M):
    V[j] = np.random.multivariate_normal(muVM[j] ,sigmaVM[j])
  VS[t] = V
  for i in range(M):
    VVT[i] = V[i, :].reshape(K, 1) @ V[i ,:].reshape(1, K)
    
  print (t + 1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


## Predictions and RMSE 

In [0]:
# Predicting Ratings using learned latent feautures
predictedR = np.zeros((N ,M))
for i in range(T):
  predictedR += US[i] @ VS[i].T
predictedR /= T
predCheck = ITest * predictedR

In [0]:
# Checking for Rating overflow( > 5) or underflow( < 1 )
for i in range(N):
  for j in range(M):
    if(predCheck[i][j] > 5):
      predCheck[i][j] = 5
    elif(predCheck[i][j] < 1 and predCheck[i][j] != 0):
      predCheck[i][j] = 1

In [0]:
# Calculating the RMSE error
RMSE = (np.sum((predCheck - RTest) ** 2) / np.sum(ITest)) ** 0.5
RMSE

0.9162960758121094